In [6]:
import os
import sys
import numpy as np
import itertools
import pickle as pkl
import h5py

In [ ]:
!ls /global/project/projectdirs/dayabay/data/exp/dayabay/2014/p14b/Neutrino

0101  0127  0222  0320	0415  0511  0606  0703	0729  0824  0919  1015	1110
0102  0128  0223  0321	0416  0512  0607  0704	0730  0825  0920  1016	1111
0103  0129  0224  0322	0417  0513  0608  0705	0731  0826  0921  1017	1112
0104  0130  0225  0323	0418  0514  0609  0706	0801  0827  0922  1018	1113
0105  0131  0226  0324	0419  0515  0610  0707	0802  0828  0923  1019	1114
0106  0201  0227  0325	0420  0516  0611  0708	0803  0829  0924  1020	1115
0107  0202  0228  0326	0421  0517  0612  0709	0804  0830  0925  1021	1116
0108  0203  0301  0327	0422  0518  0613  0710	0805  0831  0926  1022	1117
0109  0204  0302  0328	0423  0519  0614  0711	0806  0901  0927  1023	1118
0110  0205  0303  0329	0424  0520  0615  0712	0807  0902  0928  1024	1119
0111  0206  0304  0330	0425  0521  0616  0713	0808  0903  0929  1025	1120
0112  0207  0305  0331	0426  0522  0617  0714	0809  0904  0930  1026	1121
0113  0208  0306  0401	0427  0523  0618  0715	0810  0905  1001  1027	1122
0114  0209  0307  0402	0428  0524  061

## Access Issues
Two Directories have files that can't be accessed: 

* **/project/projectdirs/dayabay/data/exp/dayabay/2013/p14a/**
    * has some empty directories
    * these seem to all be regenerated in /project/projectdirs/dayabay/data/exp/dayabay/2013/p15a/Neutrino/

* **/project/projectdirs/dayabay/data/exp/dayabay/2012/p14a**
    * sym links to /global/projecta/projectdirs/dayabay/data/exp/dayabay/2012/p14a
    * permission denied
    
## Solution
All unfound files except /project/projectdirs/dayabay/data/exp/dayabay/2012/p14a/Neutrino/1105/recon.Neutrino.0034955.Physics.EH3-Merged.P14A-P._0025.root can be found in /project/projectdirs/dayabay/data/exp/dayabay/year/p15a


In [7]:
def get_id(file_string):
    fs_split = file_string.split('.')
    return fs_split[2] + fs_split[6] + fs_split[4]

In [8]:
def get_filepaths(directory,filter_fxn=None):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames). Only takes files that make
    filter_fxn evaluate to true
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            if filter_fxn is not None:
                if not filter_fxn(filename):
                    continue
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

In [9]:
def get_eh(rootfile):
    fs_split = rootfile.split('.')
    return fs_split[4].split('-')[0]

In [10]:
with open("./FileList-6Oct-Official-1") as f:
    content = [x.strip('\n') for x in f.readlines()]
with(open("./FileList-6Oct-Official-2")) as f:
    content2 = [x.strip('\n') for x in f.readlines()]
new_content = content + content2
new_content = list(set(new_content))

In [11]:
print "official list 1 has %i elemeents, official list \
2 has %i elements. Their sum is %i and the set of the union has %i elements" %\
(len(content), len(content2), len(content) + len(content2),  len(new_content))

official list 1 has 189159 elemeents, official list 2 has 118961 elements. Their sum is 308120 and the set of the union has 308120 elements


In [12]:
eh1 = [c for c in new_content if get_eh(c) == 'EH1' ]

In [13]:
len(eh1)

137544

In [14]:
old_content = content
content = new_content


In [15]:
#list of files in content that can cannot be read
cant = [ f for f in content if not os.path.exists(f) ]
can = [ f for f in content if os.path.exists(f) ]

In [11]:
print can[0]
print cant[1]
print len(cant)
print len(can)

/project/projectdirs/dayabay/data/exp/dayabay/2012/p14a/Neutrino/0103/recon.Neutrino.0021367.Physics.EH1-Merged.P14A-P._0092.root
/project/projectdirs/dayabay/data/exp/dayabay/2013/p14a/Neutrino/0422/recon.Neutrino.0038701.Physics.EH1-Merged.P14A-P._0026.root
261585
46535


In [39]:
cant_2014 = [c for c in cant if "2014" in c] #.split("Neutrino/")[1].split("/recon")[0]

In [40]:
print cant_2014[0]

/project/projectdirs/dayabay/data/exp/dayabay/2014/p14b/Neutrino/0103/recon.Neutrino.0043809.Physics.EH2-Merged.P14B-P._0064.root


In [16]:
print "I cant read files from these years %s" % str(list(set([c.split('/')[7] for c in cant ])))
print "I can read files from these years %s" % str(list(set([c.split('/')[7] for c in can ])))

I cant read files from these years ['2014', '2013', '2012']
I can read files from these years ['1970', '2015', '2014', '2011', '2013', '2012']


In [17]:
can_ids = [get_id(c) for c in can]
cant_ids = [get_id(c) for c in cant]

In [30]:
print can_ids[0]

0021367_0092EH1-Merged


In [18]:
new_base = '/global/projecta/projectdirs/dayabay/data/exp/dayabay/'
#old_base = 
new_2013 = os.path.join(new_base, "2013/p14b/Neutrino/")#'/project/projectdirs/dayabay/data/exp/dayabay/2013/p15a/Neutrino/'
new_2012 = os.path.join(new_base, "2012/p14a/Neutrino/") #'/project/projectdirs/dayabay/data/exp/dayabay/2012/p15a/Neutrino/'
#new_2014 = os.path.join(new_base, "2014/p15a/Neutrino/")#'/project/projectdirs/dayabay/data/exp/dayabay/2014/p15a/Neutrino/'
for path in [new_2013, new_2012]:#, new_2014]:
    if os.path.exists(path):
        print "%s exists" % path
    else:
        print "%s don't exist foo" % path  
        
    

/global/projecta/projectdirs/dayabay/data/exp/dayabay/2013/p14b/Neutrino/ exists
/global/projecta/projectdirs/dayabay/data/exp/dayabay/2012/p14a/Neutrino/ exists


In [20]:
new_files_2012 = get_filepaths(new_2012,lambda f: '.root' in f) 
new_files_2013 = get_filepaths(new_2013,lambda f: '.root' in f) 
#new_files_2014 = get_filepaths(new_2014,lambda f: '.root' in f) 

In [21]:
new_files = new_files_2013 + new_files_2012# + new_files_2014 

In [30]:
new_files[100]
len(new_files)
#can[0]



43121

[]


In [31]:
new_ids = {get_id(fil): fil for fil in new_files}

In [32]:
cant_dict = {get_id(f): f for f in cant}

In [33]:
num_cant = len(cant)
num_no_need_to_recov = len(set(can_ids).intersection(set(new_ids.keys())))

In [34]:
recov = list(set(cant_ids).intersection(set(new_ids.keys())))
num_recov = len(recov)

In [35]:
not_recovered = list(set(cant_ids).difference(set(new_ids.keys())))
print "Number of files I can't read: %i, total recoveredthat were needed: %i. recov that I could already read anyway %i " % (num_cant, num_recov, num_no_need_to_recov)
print "Total files on list: %i, Total files recovered from 2012,2013,2014: \
%i, Files recov not on list: %i" % (len(content), len(new_files), len(new_files) - num_recov - num_no_need_to_recov)
#print not_recovered

Number of files I can't read: 261585, total recoveredthat were needed: 662. recov that I could already read anyway 42557 
Total files on list: 308120, Total files recovered from 2012,2013,2014: 43121, Files recov not on list: -98


In [36]:
files_not_recovered = [f for f in cant if get_id(f) in not_recovered]
print "file not recovered: ", files_not_recovered
print len(content)
print len(new_ids)

KeyboardInterrupt: 

In [84]:
files_recovered =  [new_ids[f_id] for f_id in recov]

In [85]:
len(files_recovered)

261585

In [86]:
new_official_list = files_recovered + can

In [87]:
len(new_official_list)

308120

In [88]:
with open('./FileList-14Mar-Recovered-1-2', 'w') as f:
    for item in new_official_list:
        f.write("%s\n" % item)

In [89]:
!ls


FileList-14Mar-Recovered-1-2  extract_background.ipynb
FileList-6Oct-Official-1      makedataset_withtime.py
FileList-6Oct-Official-2      write_new_filelist.ipynb


In [90]:
with open('./FileList-14Mar-Recovered-1-2', 'r') as f:
    test_content = [x.strip('\n') for x in f.readlines()]

In [91]:
for i, c in enumerate(test_content):
    assert test_content[i] == new_official_list[i], "Ahhh sopmething went wrong!"